# Analyse de l'influence des distributions des temps inter-véhiculaires sur un axe routier

Paramètres de simulation:
- 1 voie
- longueur de la voie : 1900 m
- présence d'outils de contrôle de la circulation : non
- paramètres de distribution 
    <br/> dn -> N(8.33, 1) [m]
    <br/> longueur -> N(7, 1) [m]
    <br/> vitesse -> N(14, 3) [m.s^-1]
    <br/> t -> N(2, 0.5) [s]
- durée de la simulation : 500 s = 5000 1/10s   

import des librairies:

In [ ]:
import numpy as np  
import network, simulation, toolkit
import makesimulation 
import matplotlib.pyplot as plt
import analysis
import statistics
import seaborn
from trafficintelligence import events, moving
import matplotlib.patches as mpatches

Récupération des fichiers initiaux

In [ ]:
world = network.World.load('simple-net.yml')  
sim = simulation.Simulation.load('config.yml')

- headways -> Exp(1.8, 1)
</br> On décalle la distribution à 1s % les temps inter-véhiculaires produits ne seront pas inférieurs à cette valeur

In [ ]:
import numpy as np  
import network, simulation, toolkit
import makesimulation 
import matplotlib.pyplot as plt
import analysis
import statistics
import seaborn
from trafficintelligence import events, moving


world = network.World.load('simple-net.yml')  
sim = simulation.Simulation.load('config.yml')
sim.duration = 100
seeds = [52]#,33, 0,110,2,45]
headways = [1.5]#, 1.8, 2,2.4, 3]
interactions = {}
usersCount = {}
for seed in seeds : 
    usersCount[seed] = {}
    interactions[seed] = {}
    for h in headways:
    
        interactions[seed][h] = []
        world = network.World.load('simple-net.yml')  
        sim.seed = seed
        world.userInputs[0].distributions['headway'].scale = h-world.userInputs[0].distributions['headway'].loc
        
        world = makesimulation.run(world, sim)
        usersCount[seed][h] = world.getNotNoneVehiclesInWorld()
        for k in range(len(world.userInputs[0].alignment.users)-1):
            roadUser1 = world.userInputs[0].alignment.users[k]
            roadUser2 = world.userInputs[0].alignment.users[k+1]
            if roadUser1.timeInterval is not None and roadUser2.timeInterval is not None:
                i = events.Interaction(useCurvilinear=True, roadUser1=roadUser1, roadUser2=roadUser2)
                i.computeIndicators(world=world, alignment1=world.travelledAlignments(roadUser1), alignment2=world.travelledAlignments(roadUser2)) 
                interactions[seed][h].append(i)
toolkit.callWhenDone()

# 1-1) - distance d'interaction minimale

In [ ]:
minDistanceList = {} # liste des distances minimales pour chaque simulation, pour chaque headway testé
minDistances = []
for h in headways:
    minDistanceList[h] = []
    for seed in seeds:
        for inter in interactions[seed][h]:
            minDistances.append(min(inter.indicators['Distance'].values.values()))
        minDistanceList[h].append(np.mean(minDistances))
        
data = [minDistanceList[x] for x in minDistanceList]

# boxplot 
plt.figure()
plt.boxplot(data, vert=True, labels=headways) 
plt.xlabel('headway (s)')
plt.ylabel('average minimal interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/boxplot/distance_interactions_min/distance_interactions_min.png')   
plt.figure()
# courbe des moyennes
toDraw = []
for val in data:
    toDraw.append(np.mean(val))
plt.plot(headways, toDraw)
plt.xlabel('headway (s)')
plt.ylabel('average minimal interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/distance_interactions_min/distance_interactions_min.png')   

plt.close('all')
##### OK ####

# 1-2) Distance d'interaction moyenne

minDistanceList = {} # liste des distances minimales pour chaque simulation, pour chaque headway testé
minDistances = []
import statistics
for h in headways:
    minDistanceList[h] = []
    for seed in seeds:
        for inter in interactions[seed][h]:
            minDistances.append(statistics.mean(inter.indicators['Distance'].values.values()))
        minDistanceList[h].append(np.mean(minDistances))
        
data = [minDistanceList[x] for x in minDistanceList]
plt.figure()


# boxplot 
plt.boxplot(data, vert=True, labels=headways) 
plt.xlabel('headway (s)')
plt.ylabel('average mean interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/boxplot/distance_interactions_mean/distance_interactions_mean.png')   

plt.figure()
# courbe des moyennes
toDraw = []
for val in data:
    toDraw.append(np.mean(val))
plt.plot(headways, toDraw)
plt.xlabel('headway (s)')
plt.ylabel('average mean interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/distance_interactions_mean/distance_interactions_mean.png')

plt.close('all')


##### OK ####

# 1-3) Distance d'interaction maximale

minDistanceList = {} # liste des distances minimales pour chaque simulation, pour chaque headway testé
minDistances = []
for h in headways:
    minDistanceList[h] = []
    for seed in seeds:
        for inter in interactions[seed][h]:
            minDistances.append(max(inter.indicators['Distance'].values.values()))
        minDistanceList[h].append(np.mean(minDistances))
        
data = [minDistanceList[x] for x in minDistanceList]

plt.figure()
# boxplot 
plt.boxplot(data, vert=True, labels=headways) 
plt.xlabel('headway (s)')
plt.ylabel('average maximal interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/boxplot/distance_interactions_max/distance_interactions_max.png')


plt.figure()
# courbe des moyennes
toDraw = []
for val in data:
    toDraw.append(np.mean(val))
plt.plot(headways, toDraw)
plt.xlabel('headway (s)')
plt.ylabel('average maximal interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/distance_interactions_max/distance_interactions_max.png')

plt.close('all')


     
##### OK ####

# 2 -  durée des interactions selon la distance seuil fixée

- par distance

distances = [20, 25, 30]
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
# fig.num=0


duration = {} # liste des durees d'interactions pour chaque simulation, pour chaque headway testé
colors = ['lightblue', 'lightgreen', 'pink']
edgesColor = ['blue', 'green', 'red']
for edgeCol, col, distance in zip(edgesColor, colors, distances):    
    for h in headways:
        interactionDuration = []
        duration[h] = []
        for seed in seeds: 
            for inter in interactions[seed][h]:
                interactionDummies = []
                for t in list(inter.timeInterval):
                    if inter.indicators['Distance'].values[t] <= distance:#min(inter.indicators['Distance'].values.values()):
                        interactionDummies.append(1)
                    else:
                        interactionDummies.append(0)
                interactionDuration.append(sum(toolkit.makeSubListFromList(interactionDummies, 1)))
            duration[h].append(np.mean(interactionDuration))
    data = [duration[x] for x in duration]
    
    toolkit.drawBoxPlot(data, edgeCol, col)
    patch = mpatches.Patch(color=col, label='d <' + str(distance)+'m')
    plt.legend(handles=[patch], loc='best')
    plt.ylabel('durée moyenne des interactions à distance<seuil  (s/10)')
    plt.xlabel('temps inter-véhiculaire moyen (s)')
    plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/boxplot/duree_interactions/'+str(distance))


    toPlot = []
    plt.figure()
    for val in data:
        toPlot.append(np.mean(val))
    plt.plot(headways, toPlot)
    plt.legend(['d <'+str(distance)+'m'])
    plt.ylabel('durée moyenne des interactions à distance<seuil  (s/10)')
    plt.xlabel('temps inter-véhiculaire moyen (s)')
    plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/duree_interactions/'+str(distance))




plt.close('all')


# plt.savefig('/home/lionel/projetmaitrise/outputData/average_duration_of_interactions.png')


- comparaison 

distances = [20, 25, 30]

duration = {} # liste des durees d'interactions pour chaque simulation, pour chaque headway testé
colors = ['lightblue', 'lightgreen', 'pink']
edgesColor = ['blue', 'green', 'red']
for edgeCol, col, distance in zip(edgesColor, colors, distances):    
    for h in headways:
        interactionDuration = []
        duration[h] = []
        for seed in seeds: 
            for inter in interactions[seed][h]:
                interactionDummies = []
                for t in list(inter.timeInterval):
                    if inter.indicators['Distance'].values[t] <= distance:#min(inter.indicators['Distance'].values.values()):
                        interactionDummies.append(1)
                    else:
                        interactionDummies.append(0)
                interactionDuration.append(sum(toolkit.makeSubListFromList(interactionDummies, 1)))
            duration[h].append(np.mean(interactionDuration))
    data = [duration[x] for x in duration]
    
    toPlot = []
    for val in data:
        toPlot.append(np.mean(val))
    plt.plot(headways, toPlot)
    plt.legend(distances)
    plt.ylabel('durée moyenne des interactions à distance<seuil  (s/10)')
    plt.xlabel('temps inter-véhiculaire moyen (s)')

plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/duree_interactions/comparaison')


plt.close('all')


# plt.savefig('/home/lionel/projetmaitrise/outputData/average_duration_of_interactions.png')



# 3 - duree d'interaction / durée de poursuite selon la distance seuil fixée

- par distance seuil

distances = [20, 25, 30]
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
# fig.num=0

colors = ['lightblue', 'lightgreen', 'pink']
edgesColor = ['blue', 'green', 'red']

proportion = {} # liste des proportions de temps ou deux vehicules sont tels que d=dmin, pour chaque simulation, pour chaque headway testé

for edgeCol, col, distance in zip(edgesColor, colors, distances):    
    for h in headways:
        interactionDuration = []
        proportion[h] = []
        for seed in seeds: 
            for inter in interactions[seed][h]:
                interactionDummies = []
                for t in list(inter.timeInterval):
                    if inter.indicators['Distance'].values[t] <= distance:# min(inter.indicators['Distance'].values.values()):
                        interactionDummies.append(1)
                    else:
                        interactionDummies.append(0)
                interactionDuration.append(sum(toolkit.makeSubListFromList(interactionDummies, 1))/len(list(inter.timeInterval)))
            proportion[h].append(np.mean(interactionDuration))
    data = [proportion[x] for x in proportion] 
    
    toolkit.drawBoxPlot(data, edgeCol, col)
    patch = mpatches.Patch(color=col, label='d <' + str(distance)+'m')
    plt.legend(handles=[patch], loc='best')
    plt.ylabel("duree d'interaction / durée de poursuite à distance<seuil  (%)")
    plt.xlabel('temps inter-véhiculaire moyen (s)')    
    plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/boxplot/duree_interaction_rapport/'+str(distance))


    toPlot = []
    plt.figure()
    for val in data:
        toPlot.append(np.mean(val))
    plt.plot(headways, toPlot)
    plt.legend(['d <'+str(distance)+'m'])
    plt.ylabel("duree d'interaction / durée de poursuite à distance<seuil  (%)")
    plt.xlabel('temps inter-véhiculaire moyen (s)')
    plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/duree_interaction_rapport/'+str(distance))
plt.close('all')


    # plt.savefig('/home/lionel/projetmaitrise/outputData/average_proportion_of_interaction_time_d=dmin.png')
# ### OK ###

- comparaison

distances = [20, 25, 30]
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
# fig.num=0

colors = ['lightblue', 'lightgreen', 'pink']
edgesColor = ['blue', 'green', 'red']

proportion = {} # liste des proportions de temps ou deux vehicules sont tels que d=dmin, pour chaque simulation, pour chaque headway testé

for edgeCol, col, distance in zip(edgesColor, colors, distances):    
    for h in headways:
        interactionDuration = []
        proportion[h] = []
        for seed in seeds: 
            for inter in interactions[seed][h]:
                interactionDummies = []
                for t in list(inter.timeInterval):
                    if inter.indicators['Distance'].values[t] <= distance:# min(inter.indicators['Distance'].values.values()):
                        interactionDummies.append(1)
                    else:
                        interactionDummies.append(0)
                interactionDuration.append(sum(toolkit.makeSubListFromList(interactionDummies, 1))/len(list(inter.timeInterval)))
            proportion[h].append(np.mean(interactionDuration))
            
    data = [proportion[x] for x in proportion] 
    toPlot = []
    for val in data:
        toPlot.append(np.mean(val))
    plt.plot(headways, toPlot)
    plt.legend(['d <'+str(distance)+'m'])
    plt.ylabel("duree d'interaction / durée de poursuite à distance<seuil  (%)")
    plt.xlabel('temps inter-véhiculaire moyen (s)')
    
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/duree_interaction_rapport/comparaison.png')


plt.close('all')


    ### ok ###

# 4) interactions / vehicule

- par distance

count = {} # liste des nombre inter/nombre vehicles quand d<seuil, pour chaque simulation, pour chaque headway testé
distances = [20,25,30]
for edgeCol, col, distance in zip(edgesColor, colors, distances):    
    for h in headways:
        count[h] = []
        for seed in seeds:
            _temp = []
            for inter in interactions[seed][h]:
                dummy = []
                for t in list(inter.timeInterval):
                    if inter.indicators['Distance'].values[t] <= distance:
                        dummy.append(1)
                    else:
                        dummy.append(0)
                c = toolkit.countElementInList(dummy, 1)
                _temp.append(c)
            count[h].append((sum(_temp)/ (len(usersCount[seed][h][0])-10)))
            
    data = [count[x] for x in count]  
    toolkit.drawBoxPlot(data, edgeCol, col)
    patch = mpatches.Patch(color=col, label='d <' + str(distance)+'m')
    plt.legend(handles=[patch], loc='best')
    plt.ylabel("Nombre d'interaction / Nombre de vehicule  (%)")
    plt.xlabel('temps inter-véhiculaire moyen (s)')   
    plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/boxplot/interaction_par_vehicule/'+str(distance))




    toPlot = []
    plt.figure()
    for val in data:
        toPlot.append(np.mean(val))
    plt.plot(headways, toPlot)
    plt.legend(['d <'+str(distance)+'m'])
    plt.ylabel("Nombre d'interaction / Nombre de vehicule  (%)")
    plt.xlabel('temps inter-véhiculaire moyen (s)')
    plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/interaction_par_vehicule/'+str(distance))


plt.close('all')


### ok ###

- comparaison

count = {} # liste des nombre inter/nombre vehicles quand d<seuil, pour chaque simulation, pour chaque headway testé
distances = [20,25,30]
for edgeCol, col, distance in zip(edgesColor, colors, distances):    
    for h in headways:
        count[h] = []
        for seed in seeds:
            _temp = []
            for inter in interactions[seed][h]:
                dummy = []
                for t in list(inter.timeInterval):
                    if inter.indicators['Distance'].values[t] <= distance:
                        dummy.append(1)
                    else:
                        dummy.append(0)
                c = toolkit.countElementInList(dummy, 1)
                _temp.append(c)
            count[h].append((sum(_temp)/ len(usersCount[seed][h][0])))
            
    data = [count[x] for x in count] 
    toPlot = []
    for val in data:
        toPlot.append(np.mean(val))
    plt.plot(headways, toPlot)
    plt.legend(distances)
    plt.ylabel("Nombre d'interaction / Nombre de vehicule  (%)")
    plt.xlabel('temps inter-véhiculaire moyen (s)')
    
plt.savefig('/home/lionel/projetmaitrise/outputData/ligne/courbe/interaction_par_vehicule/comparaison.png')



plt.close('all')

  
### ok ###

#  Analyse de l'influence des distributions des temps inter-véhiculaires sur un axe routier

import numpy as np  
import network, simulation, toolkit
import makesimulation 
import matplotlib.pyplot as plt
import analysis
import statistics
import seaborn
from trafficintelligence import events, moving


world = network.World.load('cross-net.yml')  
sim = simulation.Simulation.load('config.yml')

seeds = [52]# ,33]# 0,110,2,45]
headways = [2]#, 1.8]#, 2,2.4, 3]
interactions = {}
usersCount = {}
c=0
for seed in seeds : 
    print(seed)
    usersCount[seed] = {}
    interactions[seed] = {}
    for h in headways:
        print(h)
        interactions[seed][h] = []
        world = network.World.load('cross-net.yml')  
        sim.seed = seed
        world.userInputs[0].distributions['headway'].scale = h-world.userInputs[0].distributions['headway'].loc
        world.userInputs[1].distributions['headway'].scale = h-world.userInputs[1].distributions['headway'].loc


        world = makesimulation.run(world, sim)
        usersCount[seed][h] = world.getNotNoneVehiclesInWorld()
        for generatedUserByFistUI in range(len(world.userInputs[0].alignment.users)):
            roadUser1 = world.userInputs[0].alignment.users[generatedUserByFistUI]
            for generatedUserBySecondUI in range(len(world.userInputs[1].alignment.users)):              
                roadUser2 = world.userInputs[1].alignment.users[generatedUserBySecondUI]
                if roadUser1.timeInterval is not None and roadUser2.timeInterval is not None:
                    i = events.Interaction(useCurvilinear=True, roadUser1=roadUser1, roadUser2=roadUser2)
                    i.computeIndicators(world=world, alignment1=world.travelledAlignments(roadUser1), alignment2=world.travelledAlignments(roadUser2)) 
                    interactions[seed][h].append(i)
                    c+=1
                    print(c)
toolkit.callWhenDone()

# 1-1) - distance d'interaction minimale

minDistanceList = {} # liste des distances minimales pour chaque simulation, pour chaque headway testé
minDistances = []
for h in headways:
    minDistanceList[h] = []
    for seed in seeds:
        for inter in interactions[seed][h]:
            minDistances.append(min(inter.indicators['Distance'].values.values()))
        minDistanceList[h].append(np.mean(minDistances))
        
data = [minDistanceList[x] for x in minDistanceList]

# boxplot 
plt.figure()
plt.boxplot(data, vert=True, labels=headways) 
plt.xlabel('headway (s)')
plt.ylabel('average minimal interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/croisement/boxplot/distance_interactions_min/distance_interactions_min.png')   
plt.figure()
# courbe des moyennes
toDraw = []
for val in data:
    toDraw.append(np.mean(val))
plt.plot(headways, toDraw)
plt.xlabel('headway (s)')
plt.ylabel('average minimal interaction distance (m)')
plt.savefig('/home/lionel/projetmaitrise/outputData/croisement/courbe/distance_interactions_min/distance_interactions_min.png')   

plt.close('all')
##### OK ####